Uploading data from my Drive

In [1]:
# ! gdown https://drive.google.com/drive/folders/1eJPlwS6bd_3SUlm-rh_t7KglTaR4zrIQ?usp=sharing

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Importing libraries

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import random

Instructions for updating:
non-resource variables are not supported in the long term


loading data model from CNN model and normilze the array(balck and white) 

In [3]:
x_train = np.load("/content/gdrive/MyDrive/saved_data/x_train.npy", allow_pickle=True)
x_test = np.load("/content/gdrive/MyDrive/saved_data/x_test.npy", allow_pickle=True)
y_train = np.load("/content/gdrive/MyDrive/saved_data/y_train.npy", allow_pickle=True)
y_test = np.load("/content/gdrive/MyDrive/saved_data/y_test.npy", allow_pickle=True)

x_train = x_train/255.
x_test = x_test/255.

adding algorithm parameters

In [4]:
# Parameters
learning_rate = 0.0005
training_epochs = 80
batch_size = 256
display_step = 1

In [5]:
def next_batch(num_split, feature, labels):
    ind = np.arange(0, len(feature))
    np.random.shuffle(ind)
    ind = ind[:num_split]
    feature_shuffle = [feature[i].flatten() for i in ind]
    labels_shuffle = [labels[i] for i in ind]

    return np.asarray(feature_shuffle), np.asarray(labels_shuffle)


Building the Neural Network and runnig the algorithm

In [6]:
x = tf.placeholder(tf.float32, [None, 22500])
y = tf.placeholder(tf.float32, [None, 2])

W1 = tf.Variable(tf.random_normal([22500, 256], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([256]), name='b1')
z1 = tf.nn.relu(tf.matmul(x,W1)+b1)

W2 = tf.Variable(tf.random_normal([256, 128], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([128]), name='b2')
z2 = tf.nn.relu(tf.matmul(z1,W2)+b2)

W3 = tf.Variable(tf.random_normal([128, 64], stddev=0.03), name='W3')
b3 = tf.Variable(tf.random_normal([64]), name='b3')
z3 = tf.nn.relu(tf.matmul(z2,W3)+b3)

W4 = tf.Variable(tf.random_normal([64, 2], stddev=0.03), name='W4')
b4 = tf.Variable(tf.random_normal([2]), name='b4')


log = tf.matmul(z3, W4) + b4
pred = tf.nn.softmax(tf.matmul(z3, W4) + b4)


correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=log))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(x_train)/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = next_batch(batch_size, x_train, y_train)
            _, c, acc = sess.run([optimizer, cost, accuracy], feed_dict={x: batch_xs, y: batch_ys})
            avg_cost += c / total_batch
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost), "acc=", "{:.5}".format(acc))
    print("Optimization Finished!")
    
    x_test_flat = []
    for i in x_test:
        x_test_flat.append(i.flatten())
    x_test_flat = np.array(x_test_flat)
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: x_test_flat, y: y_test}))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Epoch: 0001 cost= 0.799107522 acc= 0.52344
Epoch: 0002 cost= 0.677708346 acc= 0.67188
Epoch: 0003 cost= 0.605731291 acc= 0.83203
Epoch: 0004 cost= 0.453389627 acc= 0.86719
Epoch: 0005 cost= 0.295134902 acc= 0.89844
Epoch: 0006 cost= 0.225273612 acc= 0.91406
Epoch: 0007 cost= 0.203871648 acc= 0.93359
Epoch: 0008 cost= 0.182173088 acc= 0.92969
Epoch: 0009 cost= 0.149692709 acc= 0.94141
Epoch: 0010 cost= 0.163126971 acc= 0.90625
Epoch: 0011 cost= 0.131931464 acc= 0.94922
Epoch: 0012 cost= 0.119927533 acc= 0.97266
Epoch: 0013 cost= 0.107575840 acc= 0.96094
Epoch: 0014 cost= 0.101135464 acc= 0.97656
Epoch: 0015 cost= 0.118921620 acc= 0.94922
Epoch: 0016 cost= 0.110746482 acc= 0.96875
Epoch: 0017 cost= 0.143160866 acc= 0.89844
Epoch: 0018 cost= 0.121881900 acc= 0.97266
Epoch: 0019 cost= 0.091724247 acc= 